In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Groupby apply on Overall Relational dataset

# The Functions requires 'date' feature to be sorted

In [2]:
# Reading the relational dataset into a dataframe

pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])

# Sorting the dates in ascending nature

pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()

pd_JH_data.tail()

,date,state,country,confirmed
261625,2022-07-27,no,Andorra,45326
261626,2022-07-27,no,Algeria,267096
261627,2022-07-27,no,Albania,308050
261628,2022-07-27,no,Argentina,9507562
261629,2022-07-27,no,Zimbabwe,256315


# Checking for small set of data

In [3]:
data_pool=pd_JH_data[((pd_JH_data['country']=='Sweden')|
                      (pd_JH_data['country']=='Italy'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [4]:
data_pool.head()

,date,state,country,confirmed
16872,2020-03-21,no,Italy,53578
16896,2020-03-21,no,Sweden,1868
17157,2020-03-22,no,Italy,59138
17180,2020-03-22,no,Sweden,1986
17442,2020-03-23,no,Italy,63927


In [5]:
# Applying the groupby command only on the 'country' on the small set of the data to test the results

data_pool.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Italy,2022-07-27,no,20837233
Sweden,2022-07-27,no,2533978


In [6]:


import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    '''Application of linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [7]:
data_pool.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Italy   2022-07-27   20837233
      Sweden  2022-07-27    2533978

In [8]:
# For us to for analys changing relationships among variables overtime and to get the dataset for the doubling rate we apply Rolling regression

# Rolling regressions estimate model parameters using a fixed window of time over the entire data set. A larger sample size, 
# or window, used will result in fewer parameter estimates but use more observations. 


# Defining the rolling function which is following the linear regression behaviour on the 'confirmed' feature of the relational dataset

def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [9]:
# On grouping the features 'state','country' here we apply the rolling function on the data of the 'confirmed' column from the test dataframe

data_pool[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Italy    16872           NaN
                17157           NaN
                17442     11.379070
                17728     12.767550
                18012     13.225547
                            ...    
       Sweden   260287          inf
                260571          inf
                260857          inf
                261141          inf
                261430          inf
Name: confirmed, Length: 1718, dtype: float64

In [10]:
# Applying rolling function on the entire dataset of the dataframe

pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [11]:
pd_DR_result

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1048,NaN
4,Alberta,Canada,1354,NaN
...,...,...,...,...
261625,no,Zimbabwe,260296,17670.804598
261626,no,Zimbabwe,260538,13850.756757
261627,no,Zimbabwe,260866,21354.722222
261628,no,Zimbabwe,261108,17084.622222


In [12]:
# As the 'confirmed' feature from the above dataframe is actually a confirmed doubling rate obtained by applying the rolling regression function on
# the intially existing confirmed cases. Therefore we change the name of the feature to 'confirmed_DR'

pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1048,NaN
4,Alberta,Canada,1354,NaN


In [13]:
pd_JH_data=pd_JH_data.reset_index()

pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0
1,1,2020-01-22,no,Kosovo,0
2,2,2020-01-22,no,Kuwait,0
3,3,2020-01-22,no,Kyrgyzstan,0
4,4,2020-01-22,no,Laos,0


In [14]:
# To have the new feature of 'confirmed_DR' and 'confirmed' in single dataframe, we create a new dataframe
# In the new df we merge the pd_JH_data and pd_DR_result with left approach.

pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')

pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0,NaN
1,1,2020-01-22,no,Kosovo,0,NaN
2,2,2020-01-22,no,Kuwait,0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0,NaN
4,4,2020-01-22,no,Laos,0,NaN


In [15]:
pd_result_larg[pd_result_larg['country']=='Italy']

,index,date,state,country,confirmed,confirmed_DR
58,58,2020-01-22,no,Italy,0,NaN
341,341,2020-01-23,no,Italy,0,NaN
627,627,2020-01-24,no,Italy,0,NaN
912,912,2020-01-25,no,Italy,0,NaN
1197,1197,2020-01-26,no,Italy,0,NaN
...,...,...,...,...,...,...
260263,260263,2022-07-23,no,Italy,20608190,291.650656
260548,260548,2022-07-24,no,Italy,20660065,340.398081
260833,260833,2022-07-25,no,Italy,20684182,543.499640
261118,261118,2022-07-26,no,Italy,20772833,367.226400


# Data filtering with groupby apply function

### 1] Data filtering on the Confirmed cases data

In [16]:
from scipy import signal

# First filtering on the 'confirmed'feature of the dataframe

def savgol_filter(df_input,column='confirmed',window=5):
    ''' In the groupby apply function Savgol Filter is used which 
     ensures that the data structure is kept'''
    
    window=5, 
    degree=1
    
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) 
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    
    return df_result
        

In [17]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()


In [18]:
pd_filtered_result

,index,state,country,confirmed,confirmed_filtered
0,0,Alberta,Canada,0,0.0
1,1,no,Kosovo,0,0.0
2,2,no,Kuwait,0,0.0
3,3,no,Kyrgyzstan,0,0.0
4,4,no,Laos,0,0.0
...,...,...,...,...,...
261625,261625,no,Andorra,45326,45326.0
261626,261626,no,Algeria,267096,267090.4
261627,261627,no,Albania,308050,307785.2
261628,261628,no,Argentina,9507562,9515909.0


In [19]:
# Merging the newly added feature of confirmed_filtered with previously existing dataframe having other relevant features

pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()


,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0,NaN,0.0
4,4,2020-01-22,no,Laos,0,NaN,0.0


## 2] Doubling rate with filterd values

#####  First apply groupby for the relevent features
 
##### Then application of the rolling regression function on the 'confirmed_filtered'

In [20]:
# Getting the confirmed_filtered_DR data by first grouping the dataset for the state and country and then applying the rolling command over 
# confirmed_filtered data from df pd_result_larg

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.head()

,state,country,index,confirmed_filtered_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1048,NaN
4,Alberta,Canada,1354,NaN


In [21]:
pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
261625,no,Zimbabwe,260296,17081.684444
261626,no,Zimbabwe,260538,17082.648889
261627,no,Zimbabwe,260866,15530.682828
261628,no,Zimbabwe,261108,14080.926740
261629,no,Zimbabwe,261629,15255.392857


In [22]:
# Merging the newly obtained feature of confirmed_filtered_DR to main df pd_result_larg

pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')

pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
261625,261625,2022-07-27,no,Andorra,45326,inf,45326.0,-6.229558e+15
261626,261626,2022-07-27,no,Algeria,267096,2.966748e+03,267090.4,3.260177e+03
261627,261627,2022-07-27,no,Albania,308050,2.095347e+02,307785.2,3.022665e+02
261628,261628,2022-07-27,no,Argentina,9507562,inf,9515909.0,1.139039e+03
261629,261629,2022-07-27,no,Zimbabwe,256315,1.139065e+04,256307.4,1.525539e+04


In [23]:
# To have better data quality in the dataframe a condition is defined

cut_out=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(cut_out, other=np.NaN) 

In [24]:
pd_result_larg[pd_result_larg['country']=='Italy'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
260263,260263,2022-07-23,no,Italy,20608190,291.650656,20591760.4,311.302476
260548,260548,2022-07-24,no,Italy,20660065,340.398081,20652857.2,341.014600
260833,260833,2022-07-25,no,Italy,20684182,543.499640,20712500.6,342.096050
261118,261118,2022-07-26,no,Italy,20772833,367.226400,20769586.0,354.867829
261401,261401,2022-07-27,no,Italy,20837233,271.344184,20826671.4,363.833590


# Storing the final dataframe in csv format

In [25]:

pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)